# swmm_api part 2
In this Jupyter Notebook, our goal is to explore basic functionalities of `swmm_api`. You can find the documentation [here](https://markuspichler.gitlab.io/swmm_api/README.html#).

For our analyses, we will use Example2-Post.inp and Pump_Control_Model.inp, that we worked on in class. Verify your code by comparing to the .inp file

# 1. Example 1
## 1.1 import packages and .inp file

In [ ]:
import swmm_api
from swmm_api.input_file import read_inp_file, SwmmInput
from swmm_api import SwmmInput, SwmmOutput, SwmmReport
from swmm_api.run_swmm import swmm5_run_epa, swmm5_run_progress
from swmm_api.run_swmm.run_temporary import swmm5_run_temporary
from swmm_api.output_file import OBJECTS, VARIABLES
import matplotlib.pyplot as plt 

In [ ]:
# read the file 
inp = read_inp_file('Example2-Post.inp') 

## 1.2 Run 2-yr storm simulation

In [ ]:
# check current setting
inp.OPTIONS['FLOW_ROUTING']

In [ ]:
inp.RAINGAGES['RainGage'].timeseries

In [ ]:
# check which storm events are available
inp.TIMESERIES.frame

In [ ]:
# set to 2-yr
inp.RAINGAGES['RainGage'].timeseries = '2-yr'

In [ ]:
# run simulation
with swmm5_run_temporary(inp.copy(), run = swmm5_run_progress, label='example_run_swmm') as res:
    out = res.out  # type: SwmmOutput
    rpt = res.rpt  # type: SwmmReport

In [ ]:
# # 4. Plotting results
# Select some output time series variables, such as subcatchment runoff, link flow, outfall total inflow to plot
# build in functions

plt.figure() 
out.get_part(OBJECTS.NODE, 'O1', VARIABLES.NODE.TOTAL_INFLOW).plot()
out.get_part(OBJECTS.SYSTEM, None, VARIABLES.SYSTEM.RUNOFF).plot()
plt.xlabel('Time (hours)')
plt.ylabel('CFS')
plt.title('System inflow/outflow - 2-yr')
plt.grid(True)
plt.legend()

## 1.3 Custom data and plots

In [ ]:
# save specific variable store as dataframe so you can use it later

system_outflow = out.get_part(OBJECTS.NODE, 'O1', VARIABLES.NODE.TOTAL_INFLOW).to_frame()
system_inflow = out.get_part(OBJECTS.SYSTEM, None, VARIABLES.SYSTEM.RUNOFF).to_frame()

In [ ]:
# custom plot

plt.figure() 
plt.plot(system_inflow, color='blue', label='Inflow', linewidth=2, alpha=0.5)
plt.plot(system_outflow, color='red', label='Outflow', linewidth=2, alpha=0.5)
# Formatting the plot
plt.xlabel('Time')
plt.ylabel('Flow (cfs)')
plt.legend()
#plt.savefig('figures/CTown_time_series_demand.png', dpi = 400)
plt.show()


## 1.4 Compare 2-yr and 100-yr events

In [ ]:
# set 100-yr storm event

inp.RAINGAGES['RainGage'].timeseries = '100-yr'

# run simulation
with swmm5_run_temporary(inp.copy(), run = swmm5_run_progress, label='example_run_swmm') as res:
    out1 = res.out  # type: SwmmOutput
    rpt1 = res.rpt  # type: SwmmReport

### System outflow

In [ ]:
plt.figure() 
out.get_part(OBJECTS.NODE, 'O1', VARIABLES.NODE.TOTAL_INFLOW).plot()
out1.get_part(OBJECTS.NODE, 'O1', VARIABLES.NODE.TOTAL_INFLOW).plot()
plt.xlabel('Time (hours)')
plt.ylabel('CFS')
plt.title('System outflow - 2 & 100-yr')
plt.grid(True)
plt.legend()

### System inflow

In [ ]:
plt.figure() 
out.get_part(OBJECTS.SYSTEM, None, VARIABLES.SYSTEM.RUNOFF).plot()
out1.get_part(OBJECTS.SYSTEM, None, VARIABLES.SYSTEM.RUNOFF).plot()
plt.xlabel('Time (hours)')
plt.ylabel('CFS')
plt.title('System inflow - 2 & 100-yr')
plt.grid(True)
plt.legend()

### Exercise:
1. Compare kinematic wave with dynamic wave
2. Select some output time series variables, such as subcatchment runoff, link flow, outfall total inflow to plot

## 1.5 Change culvert diameter and compare results

In [ ]:
# get current C11 diameter

inp.XSECTIONS['C11'].height

In [ ]:
# set C11 to 3.5 ft and get flooding loss before and after the change

inp.XSECTIONS['C11'].height = 3.5
print(inp.XSECTIONS['C11'].height)

In [ ]:
# set storm event
inp.RAINGAGES['RainGage'].timeseries = '100-yr'

In [ ]:
# run simulation

with swmm5_run_temporary(inp.copy(), run = swmm5_run_progress, label='example_run_swmm') as res:
    out2 = res.out  # type: SwmmOutput
    rpt2 = res.rpt  # type: SwmmReport

In [ ]:
# plot total flooding loss for the two scenarios

print(rpt1.flow_routing_continuity['Flooding Loss'])

print(rpt2.flow_routing_continuity['Flooding Loss'])

In [ ]:
# plot results

plt.figure() 
out1.get_part(OBJECTS.NODE, 'O1', VARIABLES.NODE.TOTAL_INFLOW).plot()
out2.get_part(OBJECTS.NODE, 'O1', VARIABLES.NODE.TOTAL_INFLOW).plot()
plt.xlabel('Time (hours)')
plt.ylabel('CFS')
plt.title('System outflow - 100-yr')
plt.grid(True)
plt.legend()

### Custom plot

In [ ]:
system_outflow1 = out1.get_part(OBJECTS.NODE, 'O1', VARIABLES.NODE.TOTAL_INFLOW).to_frame()
system_outflow2 = out2.get_part(OBJECTS.NODE, 'O1', VARIABLES.NODE.TOTAL_INFLOW).to_frame()
# custom plot

plt.figure() 
plt.plot(system_outflow1, color='blue', label = 'C11 = 4.75', linewidth=2, alpha=0.5)
plt.plot(system_outflow2, color='red', label = 'C11 = 3.5', linewidth=2, alpha=0.5)
# Formatting the plot
plt.xlabel('Time')
plt.ylabel('Flow (cfs)')
plt.legend()
plt.show()

# 2. Example 2
In this example, we will use `'Pump_Control_Model.inp'` example. This example illustrates Rule-based Control feature for simulating real-time control. The data file applies to a combined sewer system with a diversion pump installed to help prevent local overflows. The scenario modelled contains only dry weather flow, and illustrates the use of DWF time-of-day patterns to have DWF vary diurnally. We will:

1. Examine the Controls Editor to view the rules that control the on/off status of the pump.

2. View the hourly DWF time patterns that are assigned to each node that receives DWF.

3. Run the model for a 24-hour simulation.

4. Create a time series plot of water depth at the storage node and flow in the pump on the same graph. See if the plot reflects the desired control rules.

5. Modify pump model

## 2.1 Import packages and .inp file

In [ ]:
import matplotlib.pyplot as plt
from swmm_api import SwmmInput, SwmmOutput, SwmmReport
from swmm_api.input_file.macros.plotting_map import init_empty_map_plot, add_node_map, add_link_map, add_node_labels, add_link_labels, add_labels
from swmm_api.output_file import OBJECTS, VARIABLES
from swmm_api.run_swmm import swmm5_run_progress
from swmm_api.run_swmm.run_temporary import swmm5_run_temporary

In [ ]:
inp = SwmmInput('Pump_Control_Model.inp')

In [ ]:
fig, ax = init_empty_map_plot()
add_link_map(ax, inp, add_arrows=True)
add_node_map(ax, inp)
add_node_labels(ax, inp, ha='left', y_offset=15, in_layout=False)
add_labels(ax, inp)
fig.set_dpi(70)
fig.set_size_inches(10, 10)

## 2.2 Get pump and storage info

In [ ]:
inp.STORAGE['SU1']

In [ ]:
inp.PUMPS['PUMP1']

In [ ]:
inp.CURVES['PUMP_CURVE1']

In [ ]:
# plot pump curve

pump_curve = inp.CURVES['PUMP_CURVE1'].points

flow, head = zip(*pump_curve)

# Plot
plt.figure(figsize=(6, 4))
plt.plot(flow, head, marker='o')
plt.title('Pump Curve: PUMP_CURVE1')
plt.xlabel('Depth (ft)')
plt.ylabel('Flow (CFS)')
plt.grid(True)
plt.show()

In [ ]:
# pump control

print(inp.CONTROLS.to_inp_lines())

## 2.3 Get DWF time patterns

In [ ]:
dwf = inp.PATTERNS['DWF']
dwf

In [ ]:
fig, ax = plt.subplots()
x = list(range(len(dwf.factors)))
rects = ax.bar(x, dwf.factors)
ax.set_xticks(x)
_ = ax.bar_label(rects, padding=3, rotation=90)

## 2.4 Run simulation

In [ ]:
with swmm5_run_temporary(inp.copy(), run=swmm5_run_progress, label='example_run_swmm') as res:
    out = res.out  # type: SwmmOutput
    rpt = res.rpt  # type: SwmmReport

## 2.5 Plot results

In [ ]:
# storage depth

storage_depth = out.get_part(OBJECTS.NODE, 'SU1', VARIABLES.NODE.DEPTH).to_frame()

plt.plot(storage_depth, color='blue', label = 'storage depth', linewidth=2, alpha=0.5)
plt.xlabel('Time')
plt.ylabel('Depth (ft)')
plt.legend()
plt.show()

In [ ]:
# pump flow

pump_flow = out.get_part(OBJECTS.LINK, 'PUMP1', VARIABLES.LINK.FLOW).to_frame()
plt.plot(pump_flow, color='red', label = 'pump flow', linewidth=2, alpha=0.5)
# Formatting the plot
plt.xlabel('Time')
plt.ylabel('Flow (cfs)')
plt.legend()
plt.show()

### Question:
In the beginning of the simulation, the pump turns on after water depth in the storage reaches 4ft. Why doesn't it turn off after the water level drops below 4ft?

In [ ]:
# system outflow

system_outflow = out.get_part(OBJECTS.NODE, 'KRO2005', VARIABLES.NODE.TOTAL_INFLOW).to_frame()
plt.plot(system_outflow, color='red', label = 'system outflow', linewidth=2, alpha=0.5)
# Formatting the plot
plt.xlabel('Time')
plt.ylabel('Flow (cfs)')
plt.legend()
plt.show()

## 2.6 Define new pump curve
We can see that that pump oscilates, we can try a different pump curve to minimize the number of pump on/off switches

In [ ]:
# get current curve settings

inp.CURVES['PUMP_CURVE1']

In [ ]:
from swmm_api.input_file.sections import Curve

In [ ]:
# create new pump settings

pump_curve_name = 'NEW_PUMP_CURVE'
pump_curve_type = 'PUMP4'
pump_curve_points = points = [[0, 0], [1, 0.1], [2, 0.2], [3, 0.3], [4, 0.35], [5, 0.6], [10, 0.7]]

In [ ]:
# define new pump curve

new_pump_curve = Curve(pump_curve_name, pump_curve_type, pump_curve_points)

In [ ]:
# assign the new pump curve

inp.CURVES[pump_curve_name] = new_pump_curve

In [ ]:
# verify the that the new curve was added 

inp.CURVES

In [ ]:
# verify the settings of the new pump curve

inp.CURVES['NEW_PUMP_CURVE']

In [ ]:
# get the current curve assigned to PUMP1

inp.PUMPS['PUMP1']

In [ ]:
# assign the new curve to PUMP1

inp.PUMPS['PUMP1'].curve_name = 'NEW_PUMP_CURVE'

In [ ]:
# plot the new pump curve

pump_curve = inp.CURVES['NEW_PUMP_CURVE'].points

flow, head = zip(*pump_curve)

# Plot
plt.figure(figsize=(6, 4))
plt.plot(flow, head, marker='o')
plt.title('Pump Curve: PUMP_CURVE1')
plt.xlabel('Head (ft)')
plt.ylabel('Flow (CFS)')
plt.grid(True)
plt.show()

In [ ]:
# run new simulation and get results

with swmm5_run_temporary(inp.copy(), run=swmm5_run_progress, label='example_run_swmm') as res:
    out = res.out  # type: SwmmOutput
    rpt = res.rpt  # type: SwmmReport

# storage depth

storage_depth = out.get_part(OBJECTS.NODE, 'SU1', VARIABLES.NODE.DEPTH).to_frame()

plt.plot(storage_depth, color='blue', label = 'storage depth', linewidth=2, alpha=0.5)
plt.xlabel('Time')
plt.ylabel('Depth (ft)')
plt.legend()
plt.show()

In [ ]:
# pump flow

pump_flow = out.get_part(OBJECTS.LINK, 'PUMP1', VARIABLES.LINK.FLOW).to_frame()
plt.plot(pump_flow, color='red', label = 'pump flow', linewidth=2, alpha=0.5)
# Formatting the plot
plt.xlabel('Time')
plt.ylabel('Flow (cfs)')
plt.legend()
#plt.savefig('figures/CTown_time_series_demand.png', dpi = 400)
plt.show()

### Question
Both the pump flow and storage are now operating smoothly, but the storage depth exceeds 4 ft. Why?

### Exercise: change the control rule < 3ft
Change control rule so that the pump turns on if the depth exceeds 3 ft

In [ ]:
# import Control

from swmm_api.input_file.sections import Control

In [ ]:
# get pump curve


In [ ]:
# set back to the original curve



In [ ]:
# check the current rule



In [ ]:
# check the structure of the CONTROLS object 



In [ ]:
# check the structure PUMP1A object 



In [ ]:
# get the current value



In [ ]:
# set new value



In [ ]:
# run simulation 

with swmm5_run_temporary(inp.copy(), run=swmm5_run_progress, label='example_run_swmm') as res:
    out = res.out  # type: SwmmOutput
    rpt = res.rpt  # type: SwmmReport


In [ ]:
# storage depth

storage_depth = out.get_part(OBJECTS.NODE, 'SU1', VARIABLES.NODE.DEPTH).to_frame()

plt.plot(storage_depth, color='blue', label = 'storage depth', linewidth=2, alpha=0.5)
plt.xlabel('Time')
plt.ylabel('Depth (ft)')
plt.legend()
plt.show()

The depth doesn't exceed ~3 ft